# 04_Steam_Genres

---

In [0]:
from pyspark.sql import functions as F
from pyspark.sql import Row

### Chargement du dataset

In [0]:
df_steam_flat = spark.read.json("/dbfs/FileStore/export/steam_flat_prep.json")

---

### Analyse des genres

Quels sont les genres les plus représentés ?

In [0]:
df_steam_flat.select("genre").limit(5).toPandas()

,genre
0,"Indie, Simulation, Strategy"
1,"Adventure, Free to Play, Indie"
2,"Action, Adventure, Casual, Indie, Sports, Stra..."
3,"Action, Casual, Indie"
4,"Adventure, Free to Play, Indie"


Les genres sont stockés comme une chaine de caractère, chacun séparé par une virgule.

Je vais tout d'abord en faire une liste, puis j'explode cette colonne.

In [0]:
df_steam_genre = df_steam_flat \
  .withColumn("genre_list", F.split(F.col("genre"),",")) \
    .withColumn("genre_list", F.expr("transform(genre_list, x -> trim(x))")) \
    .drop("genre")
df_steam_genre.select("genre_list").limit(5).toPandas()

,genre_list
0,"[Indie, Simulation, Strategy]"
1,"[Adventure, Free to Play, Indie]"
2,"[Action, Adventure, Casual, Indie, Sports, Str..."
3,"[Action, Casual, Indie]"
4,"[Adventure, Free to Play, Indie]"


In [0]:
df_steam_genre_exploded = df_steam_genre.withColumn("genre", F.explode("genre_list"))
display(df_steam_genre_exploded.select("genre").distinct().toPandas())

genre
Education
Massively Multiplayer
Sexual Content
Adventure
Sports
Accounting
Audio Production
Video Production
Animation & Modeling
Racing


In [0]:
display(df_steam_genre_exploded.groupBy("genre").agg(F.countDistinct("appid").alias("nb_games")).orderBy(F.desc("nb_games")).limit(10).toPandas())

genre,nb_games
Indie,39465
Action,23311
Casual,21958
Adventure,21061
Strategy,10654
Simulation,10532
RPG,9255
Early Access,6078
Free to Play,3391
Sports,2561


Databricks visualization. Run in Databricks to view.

Les genres les plus représentés sur la plateforme sont **Indie**, avec 34 465 jeux, suivis par **Action**, **Casual** et **Adventure**, qui comptent chacun autour de 20 000 jeux.

Les autres genres se situent en dessous de 10 000 jeux.

---

Existe-il des genres qui ont un meilleur rapport critiques positives/négatives ?

J'additionne les critiques positives et negatives par genre

In [0]:
df_genre_reviews = df_steam_genre_exploded.groupBy("genre").agg(
    F.sum("positive").alias("sum_positive"),
    F.sum("negative").alias("sum_negative"))

Je calcule du ratio global positif/négatif par genre

In [0]:
display(df_genre_reviews.withColumn("review_ratio", F.col("sum_positive") / F.col("sum_negative")).orderBy(F.desc("review_ratio")).limit(10).toPandas())

genre,sum_positive,sum_negative,review_ratio
Photo Editing,572664,12490,45.8497998398719
Animation & Modeling,681931,24307,28.05492245032295
Design & Illustration,657032,23658,27.772085552455827
Utilities,719255,40342,17.82893758365971
Audio Production,62815,5543,11.33231102291178
Game Development,25508,2851,8.947036127674501
Video Production,104481,12740,8.201020408163265
Indie,30495244,3920621,7.778166775110371
,183740,25941,7.082996029451447
Casual,9691298,1385023,6.997210876642482


Databricks visualization. Run in Databricks to view.

Le genre affichant le meilleur ratio critiques positives/négatives est **Photo Editing**, avec un score de 45.
Il est suivi par **Animation & Modeling** et **Design & Illustration**, qui affichent un score de 28.
Cela indique qu’ils reçoivent très peu de critiques négatives par rapport aux positives.

Cependant, pour affiner cette analyse, il serait pertinent de confronter ce ratio au nombre total de critiques afin d’évaluer la robustesse des résultats.

---

Est-ce que certains éditeurs ont des genres favoris ?

Nombre de jeux par éditeur et par genre

In [0]:
df_pub_genre = df_steam_genre_exploded.groupBy("publisher", "genre").agg(F.count("appid").alias("nb_games"))

Nombre total de jeux par éditeur

In [0]:
df_pub_total = df_steam_genre_exploded.groupBy("publisher").agg(F.count("appid").alias("total_games"))

Calcul de la part de chaque genre chez l'éditeur

In [0]:
df_pub_pref = df_pub_genre.join(df_pub_total, on="publisher").withColumn("genre_share", F.col("nb_games") / F.col("total_games"))

On garde le genre le plus représenté chez un éditeur

In [0]:
from pyspark.sql import Window

window = Window.partitionBy("publisher").orderBy(F.desc("nb_games"))

display(df_pub_pref.withColumn("rank", F.row_number().over(window)).filter(F.col("rank") == 1).drop("rank").orderBy(F.desc("genre_share")).toPandas())

publisher,genre,nb_games,total_games,genre_share
Metal Fox,Indie,1,1,1.0
WhisperGames,Adventure,1,1,1.0
0o0,Audio Production,1,1,1.0
111144447777a,Action,1,1,1.0
1SiGn Games,RPG,1,1,1.0
2 Bit Determine,Indie,2,2,1.0
20 Watt Games,Indie,1,1,1.0
2MEDYA,,1,1,1.0
3d Molier,Early Access,1,1,1.0
42tones,Audio Production,1,1,1.0


In [0]:
display(df_pub_pref.withColumn("rank", F.row_number().over(window)).filter(F.col("rank") == 1).drop("rank").filter(F.col("publisher").isin("8floor", "Big Fish Games")).orderBy(F.desc("genre_share")).toPandas())

publisher,genre,nb_games,total_games,genre_share
8floor,Casual,202,243,0.831275720164609
Big Fish Games,Casual,418,834,0.5011990407673861


Databricks visualization. Run in Databricks to view.

Il est difficile de tirer des conclusions précises, car certains éditeurs sont peu actifs et disposent d’un catalogue restreint.

Les éditeurs **8floor** et **Big Fish Games** se spécialisent principalement dans les jeux **Casual**, qui représentent respectivement 83 % et 50 % de leurs titres.

Il convient toutefois de rester prudent, car certains jeux sont associés à plusieurs genres et seront donc comptabilisés autant de fois qu’ils ont de genres associés.

---

Quels sont les genres les plus lucratifs ?

J'étudie la colonne `discount`

In [0]:
from pyspark.sql.functions import min, max

df_steam_genre_exploded.select(min("discount").alias("min_discount"), max("discount").alias("max_discount")).show()

+------------+------------+
|min_discount|max_discount|
+------------+------------+
|         0.0|        90.0|
+------------+------------+



`discount` semble être un pourcentage.

J'étudie la colonne `owners`

In [0]:
df_steam_genre_exploded.select("owners").distinct().show()

+--------------------+
|              owners|
+--------------------+
|  100,000 .. 200,000|
|1,000,000 .. 2,00...|
|20,000,000 .. 50,...|
|5,000,000 .. 10,0...|
|         0 .. 20,000|
|    20,000 .. 50,000|
|2,000,000 .. 5,00...|
|50,000,000 .. 100...|
|500,000 .. 1,000,000|
|10,000,000 .. 20,...|
|  200,000 .. 500,000|
|   50,000 .. 100,000|
|200,000,000 .. 50...|
+--------------------+



`owners` est une plage de valeurs, je vais donc utiliser la moyenne pour mes calculs

In [0]:
from pyspark.sql.functions import split, translate

owners_clean = translate(F.col("owners"), ",", "")
split_owners = split(owners_clean, " .. ")

df_steam_genre_exploded = df_steam_genre_exploded \
       .withColumn("owners_min", split_owners.getItem(0)) \
       .withColumn("owners_max", split_owners.getItem(1)) \
       .withColumn("owners_avg", (F.col("owners_min") + F.col("owners_max")) / 2)

df_steam_genre_exploded.select("owners_avg").limit(5).toPandas()

,owners_avg
0,150000.0
1,150000.0
2,150000.0
3,10000.0
4,10000.0


Calcul du coût total : prix * nombre d'utilisateurs

In [0]:
df_steam_genre_exploded = df_steam_genre_exploded.withColumn("total_price", F.col("owners_avg") * F.col("price"))

df_steam_genre_exploded.select("total_price").limit(5).toPandas()

,total_price
0,29850000.0
1,29850000.0
2,29850000.0
3,0.0
4,0.0


Calcul du prix avec la remise

In [0]:
df_steam_genre_exploded = df_steam_genre_exploded.withColumn("discount_price", \
                    (100 - F.col("discount")) * F.col("total_price") / 100)

df_steam_genre_exploded.select("discount_price").limit(5).toPandas()

,discount_price
0,5970000.0
1,5970000.0
2,5970000.0
3,0.0
4,0.0


Total des prix par genre

In [0]:
display(df_steam_genre_exploded.groupBy("genre").agg(F.sum("discount_price").alias("discount_price_sum")).orderBy(F.desc("discount_price_sum")).limit(10).toPandas())

genre,discount_price_sum
Action,3.4137965409E12
Indie,2.7738668847E12
Adventure,2.3685313084E12
RPG,1.36472824075E12
Simulation,1.3270737553E12
Strategy,1.245375694E12
Casual,6.845532868E11
Early Access,4.845338823E11
Massively Multiplayer,1.772861257E11
Racing,1.4545238335E11


Databricks visualization. Run in Databricks to view.

Le genre **Action** est le plus lucratif, générant plus de 3 000 milliards de revenus.
Il est suivi par les genres **Indie** et **Adventure**, qui enregistrent chacun environ 2 500 milliards de revenus.

Ces trois genres dominent largement le marché par rapport aux autres.